In [5]:
import praw
import pandas as pd
from datetime import datetime
from praw.models import MoreComments

# TODO  
- update script to work with praw.ini configuration 
- decide on which version to use 
- do preprocessing on text data 

# Reddit Instance

In [6]:
# Creates Reddit user instance 
# Connects to Reddit using Wisdm account


reddit = praw.Reddit(
    client_id="auD_kIwyQ1r3hfxTQEYuGw",
    client_secret="XyLGeMB1mJqoaB0lCXrk4Jtmy515AA",
    password="wisdmai1234",
    user_agent="wisdm",
    username="Dramatic-Ad-9651",
    check_for_async=False
)

In [41]:
# make sure we're connected to the api
print(reddit.user.me())

Dramatic-Ad-9651


In [8]:
#tickers 
tickers = list(set(['SPY','TSLA','PYPL', 'GME', 'VIX', 
                    'AMD', 'F', 'BBBY', 'NFLX', 'NVDA', 
                    'AAPL', 'INTC', 'FSR', 'QQQ', 'TLRY',
                     'MSFT', 'TWTR', 'SNAP', 'HOOD', 'WMT', 
                     'PTON', 'WISH', 'CPRX', 'AMC', 'SNDL', 
                     'AMZN', 'DIS', 'NIO', 'FB', 'NFLX', 'LCID', 'NVDA']))

tickers[0:5]


['VIX', 'BBBY', 'SNDL', 'AMD', 'PYPL']

In [40]:
subreddits = [ 'wallstreetbets', 'stocks',  'investing', 'finance']
subreddits

['wallstreetbets', 'stocks', 'investing', 'finance']

# Subreddit

In [11]:
for thread in subreddits:
    subreddit = reddit.subreddit(thread)
    print(subreddit)

wallstreetbets
stocks
investing
finance


In [12]:
wsb = reddit.subreddit('wallstreetbets')
wsb

Subreddit(display_name='wallstreetbets')

In [13]:
for post in wsb.search('gme', sort = 'top', time_filter = 'day', limit = 10):
    print(post.title)

…And it’s gone. I set a new personal high score last Friday, only for it all to disappear in a week. I’ll see you guys again when I save up a couple thousand dollars. I will be back! But I’m sticking to SPY weeklies next time. Meme stock weeklies have brought me nothing but pain.
I think I’m almost there
Humble GME YOLO Update part two: "Dearly Regarded."
WSB banbet lost a week ago
GameStop - GME Stock Evaluation


In [14]:
for post in wsb.search('gme', sort = 'hot', time_filter = 'day', limit = 10):
    print(post.title)

Humble GME YOLO Update part two: "Dearly Regarded."
GameStop - GME Stock Evaluation
…And it’s gone. I set a new personal high score last Friday, only for it all to disappear in a week. I’ll see you guys again when I save up a couple thousand dollars. I will be back! But I’m sticking to SPY weeklies next time. Meme stock weeklies have brought me nothing but pain.
I think I’m almost there
WSB banbet lost a week ago


* need to specify which to sort by: hot, top, or mew 
* filter on day 

# Version 1

In [43]:
posts = pd.DataFrame()

ticker = 'GME'

for post in wsb.search(ticker.lower(), sort = 'new', time_filter = 'day', limit = None):
    #check if title has stock ticker 
    if ticker.lower() not in post.title.lower(): 
        #print(post.title)
        continue 
    #check if author is not banned 
    if hasattr(post.author, 'is_suspended'):
        #print(post.author.is_suspended)
        continue

    #collect desired values 
    title_instance = {
        'TYPE': 'TITLE',
        'ticker': ticker, 
        'subreddit': 'subreddit',
        'post_title': post.title, 
        'content': post.title, 
        'upvotes': post.score, 
        #'upvote_ratio': post.upvote_ratio,
        'num_comments': post.num_comments, 
        'author_karma': post.author.comment_karma, 
        'author_verified': post.author.has_verified_email, 
        'time': datetime.fromtimestamp(post.created_utc).strftime('%Y-%m-%d'),
    }

    #create row and concat it to the df
    row = pd.DataFrame([title_instance])
    posts = pd.concat([posts, row], axis = 0, ignore_index = True)

    #checking comments
    for comment in post.comments:
        #do not want sub comments of comments 
        if isinstance(comment, MoreComments):
            continue
        if 'user report' in comment.body.lower(): 
            continue 
        comment_instance = {
            'TYPE': 'COMMENT',
            'ticker': ticker, 
            'subreddit': 'subreddit', 
            'post_title': post.title,
            'content': comment.body, 
            'upvotes': comment.score, 
            #'upvote_ratio': post.upvote_ratio,
            'num_comments': comment.replies.__len__(), 
            'author_karma': comment.author.comment_karma, 
            'author_verified': comment.author.has_verified_email, 
            'time': datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d'),
        }

        row = pd.DataFrame([comment_instance])
        posts = pd.concat([posts, row], axis = 0, ignore_index = True)



In [38]:
posts.head()

,TYPE,ticker,subreddit,post_title,content,upvotes,num_comments,author_karma,author_verified,time
0,TITLE,GME,subreddit,GameStop - GME Stock Evaluation,GameStop - GME Stock Evaluation,0,5,37401,True,2022-11-04
1,COMMENT,GME,subreddit,GameStop - GME Stock Evaluation,gtfoh,7,1,8201,True,2022-11-04
2,COMMENT,GME,subreddit,GameStop - GME Stock Evaluation,>I think you're wrong about the fundamentals. ...,10,0,296648,False,2022-11-04
3,COMMENT,GME,subreddit,GameStop - GME Stock Evaluation,This ain't no movement you fuckin mouthbreathe...,1,0,1000,True,2022-11-04
4,TITLE,GME,subreddit,"Humble GME YOLO Update part two: ""Dearly Regar...","Humble GME YOLO Update part two: ""Dearly Regar...",11,8,47883,True,2022-11-04


# Version 2

In [47]:
posts = pd.DataFrame()
comments = pd.DataFrame()

ticker = 'GME'

for post in wsb.search(ticker.lower(), sort = 'new', time_filter = 'day', limit = None):
    #check if title has stock ticker 
    if ticker.lower() not in post.title.lower(): 
        #print(post.title)
        continue 
    #check if author is not banned 
    if hasattr(post.author, 'is_suspended'):
        #print(post.author.is_suspended)
        continue

    #collect desired values 
    title_instance = {
        'ticker': ticker, 
        'subreddit': 'subreddit',
        'content': post.title, 
        'upvotes': post.score, 
        'upvote_ratio': post.upvote_ratio,
        'num_comments': post.num_comments, 
        'author_karma': post.author.comment_karma, 
        'author_verified': post.author.has_verified_email, 
        'time': datetime.fromtimestamp(post.created_utc).strftime('%Y-%m-%d'),
    }

    #create row and concat it to the df
    row = pd.DataFrame([title_instance])
    posts = pd.concat([posts, row], axis = 0, ignore_index = True)

    #checking comments
    for comment in post.comments:
        #do not want sub comments of comments 
        if isinstance(comment, MoreComments):
            continue
        if 'user report' in comment.body.lower(): 
            continue 
        comment_instance = {
            'ticker': ticker, 
            'subreddit': 'subreddit', 
            # optional can remove if no grouping by title is needed 
            'post_title': post.title,
            'content': comment.body, 
            'upvotes': comment.score, 
            'replies': comment.replies.__len__(), 
            'sticked': comment.stickied,
            'author_karma': comment.author.comment_karma, 
            'author_verified': comment.author.has_verified_email, 
            'time': datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d'),
        }

        row = pd.DataFrame([comment_instance])
        comments = pd.concat([comments, row], axis = 0, ignore_index = True)

In [48]:
posts.head()


,ticker,subreddit,content,upvotes,upvote_ratio,num_comments,author_karma,author_verified,time
0,GME,subreddit,GameStop - GME Stock Evaluation,0,0.25,6,37401,True,2022-11-04
1,GME,subreddit,"Humble GME YOLO Update part two: ""Dearly Regar...",13,0.84,8,47883,True,2022-11-04


In [49]:
comments.head()

,ticker,subreddit,post_title,content,upvotes,replies,sticked,author_karma,author_verified,time
0,GME,subreddit,GameStop - GME Stock Evaluation,gtfoh,8,1,False,8202,True,2022-11-04
1,GME,subreddit,GameStop - GME Stock Evaluation,>I think you're wrong about the fundamentals. ...,11,0,False,296656,False,2022-11-04
2,GME,subreddit,GameStop - GME Stock Evaluation,This ain't no movement you fuckin mouthbreathe...,1,0,False,1000,True,2022-11-04
3,GME,subreddit,GameStop - GME Stock Evaluation,Seems optimistic.,1,0,False,20335,True,2022-11-04
4,GME,subreddit,"Humble GME YOLO Update part two: ""Dearly Regar...",https://i.redd.it/mx76ovirckw91.png\n\nOrigina...,6,0,False,47883,True,2022-11-04


# Preprocessing 